### Import dataset

In [56]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# This block will extract all the chunks // for now we will procede with only 1
file_path = 'training_set_VU_DM.csv'
# chunksize = 1000000  # Specify the number of rows to read in each chunk
# chunks = []

# # Read the CSV file in chunks
# for chunk in pd.read_csv(file_path, chunksize=chunksize):
#     # Process each chunk (example: select specific columns or apply a function)
#     processed_chunk = chunk[['column1', 'column2']]
    
#     # Append the processed chunk to the list
#     chunks.append(processed_chunk)

# # Concatenate the processed chunks into a single DataFrame
# result_df = pd.concat(chunks)

In [3]:
# it separates the data in chunks but does not return the chunk
df = pd.read_csv(file_path) #, chunksize=chunksize)

In [4]:
df.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

In [44]:
# # obtain the chunk dataset
# df_sample = df.get_chunk()

#### Exploring each variable that could be important

# 1. Search ID

In [6]:
print('Number of different customers is ', len(df['srch_id'].unique()))
df['srch_id'].value_counts()

Number of different customers is  199795


244169    38
149527    38
223395    38
250956    38
6611      37
          ..
141076     5
170616     5
188142     5
110693     5
319453     5
Name: srch_id, Length: 199795, dtype: int64

# 2. Price USD

In [12]:
'Min price is ', df['price_usd'].min(), ' max price is ', df['price_usd'].max()

('Min price is ', 0.0, ' max price is ', 19726328.0)

In [13]:
df['price_usd'].value_counts()

99.00     76129
119.00    65077
109.00    64317
89.00     62561
129.00    56130
          ...  
566.55        1
756.07        1
745.18        1
672.06        1
818.18        1
Name: price_usd, Length: 76465, dtype: int64

### 3. Comp_rate --> If expedia had better / worse prices than competition
###    Comp_inv --> availability of the hotels in Expedia / competition
###    Comp_rate_percent_diff --> difference in perc. betwenn expedia's hotels and competition

In [18]:
df['comp1_rate'].value_counts()

 1.0    67256
 0.0    42959
-1.0     9715
Name: comp1_rate, dtype: int64

In [19]:
df['comp1_inv'].value_counts()

 0.0    122599
 1.0      5492
-1.0      1468
Name: comp1_inv, dtype: int64

In [20]:
df['comp1_rate_percent_diff'].value_counts()

9.0       11317
6.0       11082
7.0        9156
10.0       8582
15.0       8515
          ...  
5273.0        1
6300.0        1
4803.0        1
2195.0        1
2682.0        1
Name: comp1_rate_percent_diff, Length: 1830, dtype: int64

In [33]:
## Checking exactly which variables are missing and how much is missing
miss_values = []

for var in df.columns:
    if df[var].isna().any():
        miss_values.append((var, df[var].isna().sum()))

miss_values.sort(key=lambda x: x[1], reverse= True)

for column, missing_count in miss_values:
    print('For varaible: ', column, " the number of missing values is ", missing_count, 'Non-missings are {}'.format(len(df) - missing_count))

For varaible:  comp1_rate_percent_diff  the number of missing values is  4863908 Non-missings are 94439
For varaible:  comp6_rate_percent_diff  the number of missing values is  4862173 Non-missings are 96174
For varaible:  comp1_rate  the number of missing values is  4838417 Non-missings are 119930
For varaible:  comp1_inv  the number of missing values is  4828788 Non-missings are 129559
For varaible:  comp4_rate_percent_diff  the number of missing values is  4827261 Non-missings are 131086
For varaible:  gross_bookings_usd  the number of missing values is  4819957 Non-missings are 138390
For varaible:  comp7_rate_percent_diff  the number of missing values is  4819832 Non-missings are 138515
For varaible:  comp6_rate  the number of missing values is  4718190 Non-missings are 240157
For varaible:  visitor_hist_starrating  the number of missing values is  4706481 Non-missings are 251866
For varaible:  visitor_hist_adr_usd  the number of missing values is  4705359 Non-missings are 252988


# Concluson :
### In terms of competition the least values that are missing are for competitors: 2,3,5,8
### The most missings are for competitors: 1, 4, 6

### Variables that are only for the training: Position, click_bool, booking_bool, gross_booking_usd

In [39]:
df['position'].max()

40

In [40]:
df['click_bool'].value_counts()
# if value is 1, cust. clicked on it

0    4736468
1     221879
Name: click_bool, dtype: int64

In [41]:
df['booking_bool'].value_counts()
# if value is 1, cust. booked the hotel

0    4819957
1     138390
Name: booking_bool, dtype: int64

#### The price for the actuall gross booking and the price shown on the offer differs ( mean values are similar, however the outliers are quite different ( and possibly it's not only due to texes, fees and conventions)) /// maybe its currency related (although the big outliers are in USD, whereas the reasonable values for price are not specified by a currency.)

In [50]:
df['gross_bookings_usd'].min(), df['gross_bookings_usd'].max(), df['gross_bookings_usd'].mean(),\
df['price_usd'].min(), df['price_usd'].max(), df['price_usd'].mean()

(0.0, 159292.38, 386.28331591878015, 0.0, 19726328.0, 254.20958985524828)

In [45]:
df['price_usd'].max() - df['gross_bookings_usd'].max()

19567035.62

In [51]:
# Correlation between the two prices tho is low
df['price_usd'].corr(df['gross_bookings_usd'])

0.007575328286871014

# Correlations

In [63]:
# Select only numeric columns
numeric_cols = df.select_dtypes(include=['float', 'int']).columns

# Create a correlation matrix
corr = df[numeric_cols].corr()

# Set the threshold for correlation coefficient
threshold = 0.5
# Filter out the correlations below the threshold
corr_high = corr[(corr > threshold) | (corr < -threshold)]

# # Set values below the threshold to NaN
# corr_high_filtered = corr_high.mask((corr_high <= threshold)&(corr_high >= -threshold))

# # remove the NaN values
# corr_high_filtered = corr_high_filtered.dropna(how='all', axis=0).dropna(how='all', axis=1)

In [65]:
# Stack the DataFrame to create a multi-level index Series
non_nan_corr = corr_high.stack()

corr_list = []
# Iterate through the index and values of the non-NaN correlations
for index, value in non_nan_corr.iteritems():
    if (index not in corr_list) & (index[::-1] not in corr_list) & (value != 1):
        print(f"Correlation between {index[0]} and {index[1]}: {value}")
        corr_list.append(index)
        corr_list.append(index[::-1])
    else:
        continue

Correlation between comp1_inv and comp5_inv: 0.5121213979079385
Correlation between comp2_rate and comp3_rate: 0.5100666193765137
Correlation between comp3_rate_percent_diff and comp5_rate_percent_diff: 0.5906580901194612
Correlation between comp3_rate_percent_diff and comp8_rate_percent_diff: 0.6349621788609786
Correlation between comp4_rate and comp8_rate: 0.5101329329438802
Correlation between comp5_rate and comp8_rate: 0.6366047920686759
Correlation between comp5_rate_percent_diff and comp8_rate_percent_diff: 0.5083390855828661
Correlation between comp6_rate and comp7_rate: 0.5100585883051445
Correlation between click_bool and booking_bool: 0.782888613406975


<ipython-input-65-f488bc4d1006>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in non_nan_corr.iteritems():


# These 2 correlations seems significant, so should be taken into account:
### Correlation between click_bool and booking_bool: 0.782888613406975

In [68]:
# Correlation between the booking, click and the position 
position_click_corr = df['position'].corr(df['click_bool'])
position_booking_corr = df['position'].corr(df['booking_bool'])

print("Correlation between position and click_bool:", position_click_corr)
print("Correlation between position and booking_bool:", position_booking_corr)

Correlation between position and click_bool: -0.16499244508537037
Correlation between position and booking_bool: -0.1479184993886952
